<a href="https://colab.research.google.com/github/hamidriasat/-Computer-Vision-and-Deep-Learning/blob/master/Yolo_Tiny_Train_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function, division
import os
import torch
import torch.nn as nn
import pandas as pd
from skimage import io, transform
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.autograd import Variable


import  os
import cv2
import numpy as np
import random




# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

datasetPath = "Dataset/test/Truck"
datasetLabels = datasetPath+"/Label"
# In[2]:



# change Bounding Box according to new img size
def cvTest(imageToPredict,imgChannels,targetImgSize, orignalBBSize):
    #     print(imageToPredict.shape)

    # Note: flipped comparing to your original code!
    # x_ = imageToPredict.shape[0]
    # y_ = imageToPredict.shape[1]
    y_ = imageToPredict.shape[0]
    x_ = imageToPredict.shape[1]

    #     targetImgSize = 250
    x_scale = targetImgSize / x_
    y_scale = targetImgSize / y_
    #     print(x_scale, y_scale)
    img = cv2.resize(imageToPredict, (targetImgSize, targetImgSize))
    #     print(img.shape)
    img = np.array(img)

    # original frame as named values
    #     (origLeft, origTop, origRight, origBottom) = (12.716032, 0.503568, 1011.603456, 781.883739)
    (origLeft, origTop, origRight, origBottom) = (orignalBBSize[0], orignalBBSize[1], orignalBBSize[2], orignalBBSize[3] )

    x = int(np.round(origLeft * x_scale))
    y = int(np.round(origTop * y_scale))
    xmax = int(np.round(origRight * x_scale))
    ymax = int(np.round(origBottom * y_scale))
    #     drawBox([[1, 0, x, y, xmax, ymax]], img)
    return img, [x, y, xmax, ymax]

def create_ground_truth(filename):

    path = 'Dataset/test/Truck'
    filename_path = path+"/Label/"+filename+".txt"
    no_of_lines = 0
    bb_coord=[]
    norm_bb_coord=[]
    targ_img_size=448
    output_vector = np.zeros((245))
    for i in range(0,245):
        num = random.random()
        output_vector[i]=num
        if(i%5==0):
            output_vector[i] = 0

    grid = np.zeros((7,7))
    image_url = path + "/" + filename + '.jpg'
    img = cv2.imread(image_url, 3)

    org_height, org_width, org_depth = img.shape
    with open(filename_path) as f:
        for line in f:
            no_of_lines = no_of_lines + 1
            ll = line.split(' ')
            x1 = ll[1]
            y1 = ll[2]
            x2 = ll[3]
            y2 = ll[4].strip()
            bb_coord.append((x1,y1,x2,y2))
            # print(bb_coord)
    #normalize the x-y, w-h coordinates
    for i in bb_coord:
        x1,y1,x2,y2=i
        mylst = [float(i) for i in [x1, y1, x2, y2]]
        temp_img ,norm_bb = cvTest(img,org_depth,targ_img_size,mylst)
        norm_bb_coord.append(norm_bb)
        # print ("nor")
        # print(norm_bb)

    #get center of the bb
    for im in norm_bb_coord:
        x1,y1,x2,y2=im
        w=(x2-x1)
        h=(y2-y1)
        #center of bb
        x = w/2+x1
        y= h/2+y1
        x_=x
        y_=y
        grid_X=0
        while(x_ > 64):
            x_ -= 64
            grid_X+=1

        grid_Y = 0
        while (y_ > 64):
            y_ -= 64
            grid_Y += 1

        grid[grid_X,grid_Y]=1
        index=(grid_X*35)+(grid_Y*5)

        #grid[1,1]=1

        #normalize the x-y
        x = (x_ - grid_X)/64
        y = (y_ - grid_Y)/64
        w=(x2-x1)/targ_img_size
        h=(y2-y1)/targ_img_size

        output_vector[index]=1.0
        output_vector[index+1] = x
        output_vector[index+2] = y
        output_vector[index+3] = w
        output_vector[index+4] = h
    return temp_img, output_vector


# In[3]:



class OpenImagesDataset(Dataset):

    def __init__(self, image_names , transform=None):
        self.image_names = image_names
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()

        path=self.image_names[idx][:-4]


        # sample = {'image': torch.from_numpy(output).to(device) , 'target': torch.from_numpy(create_dt(path)).to(device)  }
        sample = {'image_name': self.image_names[idx] , 'target_path': path}

        if self.transform:
            sample = self.transform(sample)

        return sample
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image_name, target_path = sample['image_name'], sample['target_path']
        image , target= create_ground_truth(target_path)
        return {'image': image, 'target': target}
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, target = sample['image'], sample['target']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'tensor_image': torch.Tensor(image ).to(device),
                'target_vector': torch.Tensor(target).to(device) }


        # In[4]:




files = os.listdir(datasetPath)
imagesList = []
for item in files:
    # and len(imagesList)<20
    if not item.startswith("Label") :
        imagesList.append(item)

print("Total dataset length: ",len(imagesList))

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

#         c1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
#    m1     
        self.pool1 = nn.MaxPool2d(kernel_size=2 , stride=2)
    
#         c2
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
#         m2
        self.pool2 = nn.MaxPool2d(kernel_size=2 , stride=2)
    
#          c3
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
#         m3
        self.pool3 = nn.MaxPool2d(kernel_size=2 , stride=2)
    
    #          c4
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=0)
        self.relu4 = nn.ReLU()
#         m4
        self.pool4 = nn.MaxPool2d(kernel_size=2 , stride=2)

     #          c5
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=0)
        self.relu5 = nn.ReLU()
#         m5
        self.pool5 = nn.MaxPool2d(kernel_size=2 , stride=2)
    
     #          c6
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=0)
        self.relu6 = nn.ReLU()
#         m6
        self.pool6 = nn.MaxPool2d(kernel_size=2 , stride=1)
    
    #          c7
        self.conv7 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1)
        self.relu7 = nn.ReLU()
        
         #          c8
        self.conv8 = nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3, stride=1, padding=0)
        self.relu8 = nn.ReLU()

        self.dropout = nn.Dropout(0.5)

        self.linear_1 =  nn.Linear(  512* 7* 7 ,  5* 7* 7)


    def forward(self, input):
#         c1
        output = self.conv1(input)
        output = self.relu1(output)
#         m1
        output = self.pool1(output)
    
    #         c2
        output = self.conv2(output)
        output = self.relu2(output)
#         m2
        output = self.pool2(output)
    
    #         c3
        output = self.conv3(output)
        output = self.relu3(output)
#         m3
        output = self.pool3(output)
    
    #         c4
        output = self.conv4(output)
        output = self.relu4(output)
#         m4
        output = self.pool4(output)
    
    #         c5
        output = self.conv5(output)
        output = self.relu5(output)
#         m5
        output = self.pool5(output)
    
    #         c6
        output = self.conv6(output)
        output = self.relu6(output)
#         m6
        output = self.pool6(output)
    
     #         c7
        output = self.conv7(output)
        output = self.relu7(output)
        
         #         c8
        output = self.conv8(output)
        output = self.relu8(output)

        output = self.dropout(output)

        output = output.view(output.size(0), -1)
         #         c9
        output = self.linear_1(output)

        return output


class YOLOLoss(nn.Module):
    def __init__(self):
        super(YOLOLoss, self).__init__()
        
    def forward( self, batch_size,pred,tar):
        eps=1e-05
        total= torch.tensor(0.0 ,dtype=pred.dtype).to(device)
        for k in range(batch_size):
            lambda_coord= torch.tensor(5.0)
            lambda_noobj=torch.tensor(0.5)
            first, second, third, fourth = torch.tensor(0.0 ,dtype=pred.dtype).to(device),torch.tensor(0.0, dtype=pred.dtype).to(device),torch.tensor(0.0 ,dtype=pred.dtype).to(device),torch.tensor(0.0 ,dtype=pred.dtype).to(device)
            for i in range(int(245/5)):
                j= i*5
                if tar[k][j].item() <= 0.5:
                    obj_bit= torch.tensor(0).to(device)
                    no_obj_bit= torch.tensor(1).to(device)
                else:
                    obj_bit= torch.tensor(1).to(device)
                    no_obj_bit= torch.tensor(0).to(device)

                first.add_( torch.mul(obj_bit, torch.add( torch.pow( torch.sub(pred[k][j+1],tar[k][j+1]) ,2 )
                                                          , torch.pow( torch.sub( pred[k][j+2],tar[k][j+2])  ,2 )) ) )
                second.add_( torch.mul(obj_bit, torch.add(
                    torch.pow( torch.sub( torch.sqrt(torch.clamp(pred[k][j+3], min=eps)),
                                          torch.sqrt(torch.clamp(tar[k][j+3], min=eps))) ,2) ,
                    torch.pow( torch.sub( torch.sqrt(torch.clamp(pred[k][j+4], min=eps)),
                                          torch.sqrt(torch.clamp(tar[k][j+4], min=eps))) ,2 )))   )
                third.add_( torch.mul( obj_bit, torch.pow( torch.sub( pred[k][j],tar[k][j]) ,2 ))  )
                fourth.add_( torch.mul( no_obj_bit, torch.pow( torch.sub( pred[k][j],tar[k][j]) ,2 )) )

            first.mul_( lambda_coord)
            second.mul_(lambda_coord)
            fourth.mul_( lambda_noobj)

            first.add_( second )
            first.add_(third )
            first.add_(fourth )
            total.add_(first )
#             print(total)

#         print(total/batch_size)
        return total/batch_size


model = CNNModel()
model.load_state_dict(torch.load("/content/drive/My Drive/Yolo/410"))
model.train()
if torch.cuda.is_available():
    model.to(device)

# criteria = nn.MSELoss()
criteria= YOLOLoss(  )


learning_rate = 0.001

optimizer = torch.optim.SGD(model.parameters() , lr=learning_rate)


# In[21]:
# CUDA for PyTorch


# Parameters
batch_size=64
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 0}
max_epochs = 800
data_transform = transforms.Compose([
    Rescale(448),
    ToTensor(),
])

# Generators
training_generator = DataLoader(OpenImagesDataset(imagesList ,transform=data_transform), **params)
# # Loop over epochs
for epoch in range(411,max_epochs):
    print("==========================: "+str(epoch))
    epoch_loss=0
    #     # Training
    for local_batch in enumerate(training_generator):
        optimizer.zero_grad()

        images, targets =  local_batch[1]['tensor_image'], local_batch[1]['target_vector']

        predicted = model(Variable(images, requires_grad= True) ).to(device)
        loss = criteria (images.size()[0], predicted , targets)
        # loss = criteria (predicted , targets)
        loss.backward()
        optimizer.step()
        print(loss)
        epoch_loss += loss.item()

    if epoch% 10== 0:
        torch.save(model.state_dict(), ("/content/drive/My Drive/Yolo/"+str(epoch)))
    print(epoch_loss)
    

Total dataset length:  820
==========================: 411
tensor(0.3165, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3103, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.2755, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.2604, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.2666, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3265, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3248, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.2765, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.2885, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.2805, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3382, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.2597, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.2897, device='cuda:0', grad_fn=<DivBackward0>)
3.8138464093208313
==========================: 412
tensor(0.3657, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3011, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.2839, device='cuda:0', grad_fn=<DivBackwa